<a href="https://colab.research.google.com/github/evxiong/holdem/blob/main/holdem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Heads-Up Texas Hold'em**

**Directions**: To play the game, press the play button on each of the code blocks from top to bottom. You can adjust the small and big blinds, and starting stacks.

### Import modules and define constants

In [57]:
import random
import math
from IPython.display import clear_output


In [58]:
## MODIFIABLE CONSTANTS
SMALL_BLIND = 5                             # Small blind
BIG_BLIND = 10                              # Big blind
STARTING_STACK = 50 * BIG_BLIND             # Each player's starting # of chips


### Game mechanics

In [59]:
class Game():
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
    rankings = [
        'High Card', 'Pair', 'Two Pair', 'Three of a Kind', 'Straight',
        'Flush', 'Full House', 'Four of a Kind', 'Straight Flush'
    ]
    suits = ['C', 'H', 'S', 'D']
    values = list(range(0,52))

    def __init__(self):
        """Initialize game variables before starting."""
        self.stacks = [STARTING_STACK, STARTING_STACK]
        self.dealer = 0
        self.pot = 0
        self.hole_cards = []                    # hole cards
        self.comm_cards = []                    # community cards
        self.all_in = 0                         # whether or not a player has shoved
        self.all_in_amt = 0                     # min shove amt between two players
        self.hand_count = 1

    def display(self, hand):
        """Print the string representation of each card in hand."""
        for card in hand:
            suits_ind = int(card / 13)
            ranks_ind = card % 13
            print(self.ranks[ranks_ind] + self.suits[suits_ind], end = ' ')
        print()

    def ind_to_str(self, ind):
        """Convert the integer representation of a card to a string."""
        suits_ind = int(ind / 13)
        ranks_ind = ind % 13
        return self.ranks[ranks_ind] + self.suits[suits_ind]

    def deal(self, num_players):
        """Randomly sample (without replacement) hole and community cards."""
        dealt_inds = random.sample(self.values, num_players*2 + 5)

        self.hole_cards.clear()
        self.comm_cards.clear()

        print('\nHole Cards')
        for i in range(num_players):
            self.hole_cards.append([dealt_inds[2*i], dealt_inds[2*i + 1]])
            print('P%d: %s\t%s' % (i+1, self.ind_to_str(dealt_inds[2*i]),
                                   self.ind_to_str(dealt_inds[2*i + 1])), end = '\t')
        print()
        self.comm_cards.extend(dealt_inds[2*num_players:])

    def flop(self):
        """Reveal flop cards."""
        print('Flop:\t%s %s %s' % (self.ind_to_str(self.comm_cards[0]), self.ind_to_str(self.comm_cards[1]),
                                   self.ind_to_str(self.comm_cards[2])))

    def turn(self):
        """Reveal turn card."""
        print('Turn:\t%s %s %s %s' % (self.ind_to_str(self.comm_cards[0]), self.ind_to_str(self.comm_cards[1]),
                                      self.ind_to_str(self.comm_cards[2]), self.ind_to_str(self.comm_cards[3])))

    def river(self):
        """Reveal river card."""
        print('River:\t%s %s %s %s %s' % (self.ind_to_str(self.comm_cards[0]), self.ind_to_str(self.comm_cards[1]),
                                          self.ind_to_str(self.comm_cards[2]), self.ind_to_str(self.comm_cards[3]),
                                          self.ind_to_str(self.comm_cards[4])))

    def display_pot(self):
        """Display the amount of money currently in the pot."""
        print('\nPOT: %d' % (self.pot))

    def display_stacks(self, winner=-2, gain=0):
        """Display the amount of money in each player's stack.

        Args:
            winner: Specifies winner (-2->n/a, -1->tie, 0->P1, 1->P2).
                By default, when winner not specified, display stacks without gain.
            gain: Difference between winner's current stack and stack before hand.
        """
        print('Stacks')
        if (winner == -2):
            print('P1: %d\t\tP2: %d' % (self.stacks[0], self.stacks[1]))
        elif (winner == -1):
            print('P1: %d (+%d)\tP2: %d (+%d)' % (self.stacks[0], gain, self.stacks[1], gain))
        elif (winner == 0):
            print('P1: %d (+%d)\tP2: %d' % (self.stacks[0], gain, self.stacks[1]))
        elif (winner == 1):
            print('P1: %d \tP2: %d (+%d)' % (self.stacks[0], self.stacks[1], gain))

    def blinds(self):
        """Display blinds and update each player's stack.
        
        If a player cannot supply their SB or BB, they automatically go all-in,
        with the opponent automatically matching.
        """
        print('Hand %d\n' % (self.hand_count))
        self.display_stacks()
        if (self.dealer == 0):
            print('(d)')
        else:
            print('\t\t(d)')
        print('\nBlinds: %d/%d\n' % (SMALL_BLIND, BIG_BLIND))

        sb = False
        bb = False

        if (self.stacks[self.dealer] <= SMALL_BLIND):
            self.all_in = 1
            self.all_in_amt = self.stacks[self.dealer]
        else:
            sb = True

        if (self.stacks[1 - self.dealer] <= BIG_BLIND):
            self.all_in = 1
            self.all_in_amt = self.stacks[1 - self.dealer]
        else:
            bb = True

        if (sb and bb):
            self.stacks[self.dealer] -= SMALL_BLIND
            self.stacks[1 - self.dealer] -= BIG_BLIND
            self.pot += SMALL_BLIND + BIG_BLIND
        else:
            self.stacks[self.dealer] -= self.all_in_amt
            self.stacks[1 - self.dealer] -= self.all_in_amt
            self.pot += self.all_in_amt + self.all_in_amt

        self.display_stacks()

    def bets(self, preflop=False):
        """Handle each round of bets.

        Args:
            preflop: Whether current round of bets is before or after (default) flop.
        """
        def fold(player):
            """Handle a player folding."""
            print('\nP%d folds.' % (player + 1))
            print('====== Winner: P%d ======' % (2 - player))     # other player wins
            self.stacks[1 - player] += self.pot
            self.display_stacks(1 - player, self.pot)
            self.pot = 0
            self.dealer = 1 - self.dealer
            self.hand_count += 1
            print("Press Enter to continue...")
            input()
            clear_output()
            return

        def rais(player):
            """Handle a player raising."""
            nonlocal prev_raise_amt, diff
            while (True):
                max_raise = min(self.stacks[player] - diff, self.stacks[1 - player]) # account for all in
                min_raise = min(max_raise, max(BIG_BLIND, prev_raise_amt)) # to account for stack less than BIG_BLIND/first_raise_amt 
                try:
                    amt = int(input('Raise by (min %d, max %d): ' % (min_raise, max_raise)))
                except ValueError as e:
                    print('You must enter an integer.')
                    continue

                if (amt < min_raise or amt > max_raise):
                    print('You can only raise by an amount between %d and %d. To go all-in, raise by %d.' % (min_raise, max_raise, max_raise))
                    continue
                else:
                    break

            prev_raise_amt = amt
            if (amt == max_raise):
                print('\n****** P%d IS ALL-IN ******' % (player + 1))
                self.all_in = 1
                self.all_in_amt = max_raise + diff
                self.pot += self.all_in_amt
                round_bets[player] += self.all_in_amt
                self.stacks[player] -= self.all_in_amt
            else:
                self.pot += amt + diff
                round_bets[player] += amt + diff
                self.stacks[player] = self.stacks[player] - amt - diff
            diff = abs(round_bets[0] - round_bets[1])
            return
            
        def call(player):
            """Handle a player calling."""
            nonlocal diff
            # diff should never be greater than a player's stack size??
            self.stacks[player] -= diff
            round_bets[player] = round_bets[1 - player]
            self.pot += diff
            diff = 0
            return
        
        def bet(player):
            """Handle a player betting."""
            nonlocal diff, prev_raise_amt
            while (True):
                max_bet = min(self.stacks[player], self.stacks[1 - player]) # to simplify all-in logic
                min_bet = min(max_bet, BIG_BLIND)
                try:
                    amt = int(input('Bet (min %d, max %d): ' % (min_bet, max_bet)))
                except ValueError as e:
                    print('You must enter an integer.')
                    continue

                if (amt < min_bet or amt > max_bet):
                    print('You can only bet an amount between %d and %d. To go all-in, bet %d.' % (min_bet, max_bet, max_bet))
                    continue
                else:
                    break

            prev_raise_amt = amt
            if (amt == max_bet):
                print('\n****** P%d IS ALL-IN ******' % (player + 1))
                self.all_in = 1
                self.all_in_amt = max_bet
                self.pot += self.all_in_amt
                round_bets[player] += self.all_in_amt
                self.stacks[player] -= self.all_in_amt
            else:
                self.pot += amt
                round_bets[player] += amt
                self.stacks[player] -= amt
            diff = abs(round_bets[0] - round_bets[1])
            return


        round_bets = [0,0]
        prev_raise_amt = 0

        if (preflop):
            diff = BIG_BLIND - SMALL_BLIND
            cur_player = self.dealer            # dealer acts first pre-flop
            option = 1
            round_bets = [BIG_BLIND, SMALL_BLIND] if self.dealer else [SMALL_BLIND, BIG_BLIND]
        else:
            diff = 0
            cur_player = 1 - self.dealer        # dealer acts second post-flop
            option = 0

        first_loop = True
        has_raised = False
        checked_first = False
        while (first_loop or round_bets[0] != round_bets[1] or (not has_raised and option) or checked_first):
            checked_first = False        
            
            b_str = '' if (diff or option) else 'Bet (b)  \t'
            r_str = 'Raise (r)\t' if ((diff or option) and (self.all_in == 0)) else ''
            c_str = 'Call (c) \t' if diff else ''
            h_str = 'Check (h)\t' if not diff else ''
            f_str = 'Fold (f) \t'

            print()
            print("====== P%d's TURN ======" % (cur_player + 1))
            print("--- Round Bet Status ---")
            print("P1: %d\tP2: %d\tPOT: %d" % (round_bets[0], round_bets[1], self.pot))
            print("-------- Stacks --------")
            print("P1: %d\t\tP2: %d" % (self.stacks[0], self.stacks[1]))

            print('Available actions:')
            print('%s%s%s%s%s' % (b_str,r_str,c_str,h_str,f_str))

            if (diff > 0):
                while (True):
                    s = input()
                    if (s == 'b'):
                        print('That action is not available.')
                    elif (s == 'r'):
                        has_raised = True
                        if (self.all_in):
                            print('That action is not available.')
                        else:
                            rais(cur_player)
                            break
                    elif (s == 'c'):
                        call(cur_player)
                        break
                    elif (s == 'h'):
                        print('That action is not available.')
                    elif (s == 'f'):
                        fold(cur_player)
                        return -1
                    else:
                        print('Please enter a valid action.')

            else: # diff == 0; if option==1, allow check and raise only; otherwise, bet, check, fold
                check_option = False
                while (True):
                    s = input()
                    if (s == 'b'):
                        if (option):
                            print('That action is not available.')
                        else:
                            has_raised = True
                            bet(cur_player)
                            break
                    elif (s == 'r'):
                        if (option):
                            rais(cur_player)
                            break
                        else:
                            print('That action is not available.')
                    elif (s == 'c'):
                        print('That action is not available.')
                    elif (s == 'h'):
                        if (option):
                            check_option = True
                        checked_first = True if (first_loop) else False
                        break
                    elif (s == 'f'):
                        fold(cur_player)
                        return -1
                    else:
                        print('Please enter a valid action.')
                    
                option = 0
                if (check_option):
                    check_option = False
                    break
                    
            first_loop = False
            cur_player = 1 - cur_player

        return 0

    def round(self, num_players):
        """Play one hand."""        
        self.blinds()
        self.display_pot()
        self.deal(num_players)

        if (self.all_in == 0):
            if (self.bets(preflop=True) == -1):
                return
        print()
        self.flop()
        if (self.all_in == 0):
            if (self.bets() == -1):
                return
        print()
        self.turn()
        if (self.all_in == 0):
            if (self.bets() == -1):
                return
        print()
        self.river()        
        if (self.all_in == 0):
            if (self.bets() == -1):
                return
        print()
        self.showdown()

    def showdown(self):
        hands = []
        for i in range(len(self.hole_cards)):
            hand = self.hole_cards[i]
            hand.extend(self.comm_cards)
            self.display(hand)
            temp = list(players_best_hand(hand))
            print(temp)
            print(self.rankings[temp[0] - 1])
            print()
            hands.append(temp)
        winner, ranking = rank_hands(hands)

        if (winner == -1):
            print('============ Tie ============')
        else:
            print('====== Hand Winner: P%d ======' % (winner + 1))
        print()
        if (winner == -1):
            self.stacks[0] += int(self.pot / 2)
            self.stacks[1] += int(self.pot / 2)
            self.display_stacks(winner, int(self.pot / 2))
        else:
            self.stacks[winner] += self.pot
            self.display_stacks(winner, self.pot)
        self.pot = 0
        self.all_in = 0
        self.dealer = 1 - self.dealer
        self.hand_count += 1
        print("Press Enter to continue...")
        input()
        clear_output()

    def play(self):
        """Play the game."""
        while (self.stacks[0] and self.stacks[1]):
            self.round(2)

        if (self.stacks[0] == 0):
            print()
            print("-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
            print("-=-=-=-=-=-=- GAME WINNER: P2 -=-=-=-=-=-=-")
            print("-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
        else:
            print()
            print("-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
            print("-=-=-=-=-=-=- GAME WINNER: P1 -=-=-=-=-=-=-")
            print("-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")



### Ranking hands

In [60]:
def sort_hand(hand):
    """Sort cards in hand based on value in DESCENDING order.

    Args:
        hand: List of randomly sampled ints from 0 to 51.

    Returns:
        sorted_hand: List of ranks in descending order
        sorted_suit: List of suits corresponding to ranks in sorted_hand
    """
    sorted_hand = hand.copy()
    sorted_suit = hand.copy()

    for i in range(len(sorted_hand)):
        sorted_hand[i] = (sorted_hand[i] % 13) + 2
        sorted_suit[i] = int(sorted_suit[i] / 13)

    # Insertion sort
    i = 1
    while i < len(sorted_hand):
        x = sorted_hand[i]
        y = sorted_suit[i]
        j = i-1
        while j >= 0 and sorted_hand[j] < x:
            sorted_hand[j+1] = sorted_hand[j]
            sorted_suit[j+1] = sorted_suit[j]
            j = j-1
        sorted_hand[j+1] = x
        sorted_suit[j+1] = y
        i = i+1

    return sorted_hand, sorted_suit


In [61]:
def sequence(sorted_hand, length, increment, same_suit=False, sorted_suit=None):
    """Checks for highest possible specified sequence in sorted_hand.

    Args:
        sorted_hand: List of 7 card ranks in descending order
        length: Length of desired search sequence (ex. 5 for straight, 4 for four-of-a-kind, etc.)
        increment: Step size between consecutive ranks (1 for straight, 0 for same rank)
        same_suit: If True, cards in the sequence have same suit (used for straight flush)
        sorted_suit: List of 7 suits corresponding to sorted_hand; used if same_suit is True

    Returns:
        high_card: 0 if sequence not found, rank of highest card if straight/n-of-a-kind
    """
    count = 1
    high_card = 0

    # Check for ace-low straight first
    if (increment == 1 and sorted_hand[0] == 14):
        ace_low_straight = True
        for i in reversed(range(2,6)):
            ind = len(sorted_hand)-i+1
            if ( (not same_suit and sorted_hand[ind] != i) or (same_suit and (sorted_hand[ind] != i or (sorted_suit[0] != sorted_suit[ind]) )) ):
                ace_low_straight = False
        if (ace_low_straight):
            high_card = 5

    for i in range(len(sorted_hand) + 1 - length):
        for j in range(0, length - 1):
            # check consecutive elements of sorted_hand
            if (sorted_hand[i + j] - increment == sorted_hand[i + j + 1]):
                if (not same_suit or (same_suit and sorted_suit[i] == sorted_suit[i + j + 1])):
                    count += 1
            else:
                break
        if (count == length):
            high_card = sorted_hand[i]
            break
        count = 1
    
    return high_card


In [62]:
## Hand-ranking functions

def straight_flush(sorted_hand, sorted_suit):
    """Determines whether hand contains a straight flush.
    
    Returns:
        Rank of high card if it does, 0 if it doesn't.
    """
    return sequence(sorted_hand, 5, 1, True, sorted_suit)

def four_of_a_kind(sorted_hand):
    """Determines whether hand contains a four-of-a-kind.
    
    Returns:
        Tuple (a, b) where a is the rank of the four-of-a-kind and b is
        the rank of the kicker. If no four-of-a-kind exists, returns (0, 0).
    """
    found_four = sequence(sorted_hand, 4, 0)
    if (found_four == 0):
        return 0, 0
    
    for i in reversed(range(len(sorted_hand))):
        if (sorted_hand[i] != found_four):
            return found_four,sorted_hand[i]

def full_house(sorted_hand):
    """Determines whether hand contains a full house.
    
    Returns:
        Tuple (a, b) where a is the rank of the triple and b is
        the rank of the pair. If no full house exists, returns (0, 0).
    """
    # Get highest trio (if it exists)
    found_three = sequence(sorted_hand, 3, 0)
    if (found_three == 0):
        return 0, 0

    # Remove all found_three values from sorted_hand
    sorted_hand_minus_three = [] 
    for i in sorted_hand:
        if i != found_three:
            sorted_hand_minus_three.append(i)

    # Get highest remaining pair (if it exists)
    found_pair = sequence(sorted_hand_minus_three, 2, 0)
    if (found_pair == 0):
        return 0, 0

    return found_three, found_pair

def flush(sorted_hand, sorted_suit):
    """Determines whether hand contains a flush.
    
    Returns:
        List of card ranks in highest possible flush, or
        [0, 0, 0, 0, 0] if no flush exists.
    """
    count = 0
    flush_suit = -1
    flush_ranks = [0, 0, 0, 0, 0]

    # Identify suit of flush (if it exists)
    for i in range(4):
        for j in sorted_suit:
            if (j == i):
                count += 1
        if (count >= 5):
            flush_suit = i
            break
        count = 0

    if (flush_suit == -1):
        return flush_ranks

    # Get highest cards with flush suit
    ind = 0
    for i in range(len(sorted_suit)):
        if (flush_suit == sorted_suit[i]):
            flush_ranks[ind] = sorted_hand[i]
            ind += 1
        if (ind == 5):
            break

    return flush_ranks

def straight(sorted_hand):
    """Determines whether hand contains a straight.
    
    Returns:
        Rank of high card if it does, 0 if it doesn't.
    """
    return sequence(sorted_hand, 5, 1)

def three_of_a_kind(sorted_hand):
    """Determines whether hand contains a three-of-a-kind.
    
    Returns:
        Tuple (a, b, c) where a is the rank of the triple, b and c
        are the ranks of the kickers. If no three-of-a-kind exists,
        returns (0, 0, 0).        
    """
    found_three = sequence(sorted_hand, 3, 0)
    if (found_three == 0):
        return 0, 0, 0

    kickers = []
    for i in sorted_hand:
        if (i != found_three):
            kickers.append(i)
        if (len(kickers) == 2):
            break

    return found_three, kickers[0], kickers[1]

def two_pair(sorted_hand):
    """Determines whether hand contains two pairs.
    
    Returns:
        Tuple (a, b, c) where a is the rank of the highest pair, b is
        the rank of the second-highest pair, c is the rank of the kicker.
        If no three-of-a-kind exists, returns (0, 0, 0).        
    """
    # Get highest pair (if it exists)
    found_pair1 = sequence(sorted_hand, 2, 0)
    if (found_pair1 == 0):
        return 0, 0, 0

    # Remove all found_pair1 values from sorted_hand
    sorted_hand_minus_pair1 = [] 
    for i in sorted_hand:
        if i != found_pair1:
            sorted_hand_minus_pair1.append(i)

    # Get next highest pair (if it exists)
    found_pair2 = sequence(sorted_hand_minus_pair1, 2, 0)
    if (found_pair2 == 0):
        return 0, 0, 0

    # Get kicker
    kicker = 0
    for i in sorted_hand_minus_pair1:
        if (i != found_pair2):
            kicker = i
            break

    return found_pair1, found_pair2, kicker

def pair(sorted_hand):
    """Determines whether hand contains a pair.
    
    Returns:
        Tuple (a, b, c, d) where a is the rank of the highest pair, and b,
        c, and d are the ranks of the kickers. If no pair exists, returns
        (0, 0, 0, 0).        
    """
    found_pair = sequence(sorted_hand, 2, 0)
    if (found_pair == 0):
        return 0, 0, 0, 0
    
    kickers = []
    for i in sorted_hand:
        if (i != found_pair):
            kickers.append(i)
        if (len(kickers) == 3):
            break

    return found_pair, kickers[0], kickers[1], kickers[2]


In [63]:
def players_best_hand(hand):
    """Determines best hand out of 7 possible cards.

    Args:
        hand: List of 7 integer representations of cards (each 0 to 51),
            including player's hole cards and community cards.
    
    Returns:
        Tuple (a, b) where a is the ranking of the hand and b is a list
        of high cards and kickers necessary for tiebreaking.
    """
    sorted_hand, sorted_suit = sort_hand(hand)
    high_cards = [None, None, None, None, None]
    best_ranking = 0
    
    if (straight_flush(sorted_hand, sorted_suit)):
        best_ranking = 9
        high_cards[0] = straight_flush(sorted_hand, sorted_suit)
    elif (four_of_a_kind(sorted_hand) != (0,0)):
        best_ranking = 8
        high_cards[0], high_cards[1] = four_of_a_kind(sorted_hand)
    elif (full_house(sorted_hand) != (0,0)):
        best_ranking = 7
        high_cards[0], high_cards[1] = full_house(sorted_hand)
    elif (flush(sorted_hand, sorted_suit) != [0,0,0,0,0]):
        best_ranking = 6
        high_cards = flush(sorted_hand, sorted_suit)
    elif (straight(sorted_hand)):
        best_ranking = 5
        high_cards[0] = straight(sorted_hand)
    elif (three_of_a_kind(sorted_hand) != (0,0,0)):
        best_ranking = 4
        high_cards[0], high_cards[1], high_cards[2] = three_of_a_kind(sorted_hand)
    elif (two_pair(sorted_hand) != (0,0,0)):
        best_ranking = 3
        high_cards[0], high_cards[1], high_cards[2] = two_pair(sorted_hand)
    elif (pair(sorted_hand) != (0,0,0,0)):
        best_ranking = 2
        high_cards[0], high_cards[1], high_cards[2], high_cards[3] = pair(sorted_hand)
    else: # high card
        best_ranking = 1
        high_cards = sorted_hand[0:5]

    return best_ranking, high_cards


In [64]:
def rank_hands(hands):
    """Determine round winner based on each player's best hand.

    Args:
        hands: List of each player's best hand: [[best_ranking, high_cards], ...]

    Returns:
        The winner of the round (tie->-1, P1->0, P2->1) and the ranking of the winner's best hand.
    """
    if (hands[0][0] > hands[1][0]):
        return 0, hands[0][0]
    elif (hands[0][0] < hands[1][0]):
        return 1, hands[1][0] 

    best_ranking = hands[0][0];  # players have same best hand ranking

    def winner(num_kickers):
        """Determine winner based on kickers."""
        for i in range(num_kickers):
            if (hands[0][1][i] > hands[1][1][i]):
                return 0
            elif (hands[0][1][i] < hands[1][1][i]):
                return 1
        return -1

    if (best_ranking == 9):  # straight flush
        return winner(1), best_ranking
    elif (best_ranking == 8):  # four of a kind
        return winner(2), best_ranking
    elif (best_ranking == 7):  # full house
        return winner(2), best_ranking
    elif (best_ranking == 6):  # flush
        return winner(5), best_ranking
    elif (best_ranking == 5):  # straight
        return winner(1), best_ranking
    elif (best_ranking == 4):  # three of a kind
        return winner(3), best_ranking
    elif (best_ranking == 3):  # two pair
        return winner(3), best_ranking
    elif (best_ranking == 2):  # pair
        return winner(4), best_ranking
    else:  # high card
        return winner(5), best_ranking


### Play

In [ ]:
holdem = Game()
holdem.play()